In [61]:
import numpy as np
import pandas as pd
from collections import Counter
import seaborn as sb
import re
import matplotlib.pyplot as plt
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/test.csv")

predictors = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Family_id','Family_size','Namelength','Title']
family = data_train['Name'].apply(lambda x:x.split(',')[0])
#sorted(Counter(family).items(), key=lambda i: i[1],reverse=True)
family_count = {}
for var in family:
    if var in family_count:
        family_count[var]+=1
    else:
        family_count[var]=1
family_id = {}
count = 1
for var in family:
    if family_count[var] <= 2:
        family_id[var] = 0
    elif var not in family_id:
        family_id[var] = count
        count+=1

data_train['Family_id'] = family.apply(lambda x:family_id[x])
data_train['Family_size'] = data_train['SibSp'] + data_train['Parch']
data_train['Namelength'] = data_train['Name'].apply(lambda x:len(x))

# A function to get the title from a name
def get_title(name):
    # Use a regular expression to search for a title  
    # Titles always consist of capital and lowercase letters, and end with a period
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it
    if title_search:
        return title_search.group(1)
    return ""
titles = data_train["Name"].apply(get_title)
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10, "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2,"Dona":3}
for k,v in title_mapping.items():
    titles[titles == k] = v
data_train['Title'] = titles

### test_data
family1 = data_test['Name'].apply(lambda x:x.split(',')[0])
family_count1 = {}
for var in family1:
    if var in family_count1:
        family_count1[var]+=1
    else:
        family_count1[var]=1
family_id1 = {}
count = 1
for var in family1:
    if family_count1[var] <= 2:
        family_id1[var] = 0
    elif var not in family_id1:
        family_id1[var] = count
        count+=1

data_test['Family_id'] = family1.apply(lambda x:family_id1[x])
data_test['Family_size'] = data_test['SibSp'] + data_test['Parch']
data_test['Namelength'] = data_test['Name'].apply(lambda x:len(x))

titles1 = data_test["Name"].apply(get_title)
for k,v in title_mapping.items():
    titles1[titles1 == k] = v
data_test['Title'] = titles1
#g = sb.FacetGrid(data_train,row='Pclass',col='Sex')
#g.map(plt.hist,'Survived')
#sb.barplot(x='Embarked', y='Survived', data=embark_perc,order=['S','C','Q'])
#fig, (axis1,axis2,axis3) = plt.subplots(1,3,figsize=(15,5))
#plt.show()

In [3]:
#clean data
#train
#gen new feature

#clean data
data_train.loc[data_train['Sex'] == 'male','Sex'] = 0
data_train.loc[data_train['Sex'] == 'female','Sex'] = 1
data_train['Age'] = data_train['Age'].fillna(data_train['Age'].mean())
data_train['Embarked'] = data_train['Embarked'].fillna('S')
data_train.loc[data_train['Embarked'] == 'S','Embarked'] = 0
data_train.loc[data_train['Embarked'] == 'C','Embarked'] = 1
data_train.loc[data_train['Embarked'] == 'Q','Embarked'] = 2
#test
data_test['Age'] = data_test['Age'].fillna(data_train['Age'].mean())
data_test['Fare'] = data_test['Fare'].fillna(data_train['Fare'].mean())
data_test.loc[data_test['Sex'] == 'male','Sex'] = 0
data_test.loc[data_test['Sex'] == 'female','Sex'] = 1
data_test['Embarked'] = data_test['Embarked'].fillna('S')
data_test.loc[data_test['Embarked'] == 'S','Embarked'] = 0
data_test.loc[data_test['Embarked'] == 'C','Embarked'] = 1
data_test.loc[data_test['Embarked'] == 'Q','Embarked'] = 2


In [10]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

#clf = RandomForestClassifier(n_estimators=100,min_samples_split = 4,min_samples_leaf = 2)
clf = ExtraTreesClassifier(n_estimators=100,min_samples_split = 4,min_samples_leaf = 2)
rfc = svm.SVC()
rfc.fit(data_train[predictors],data_train['Survived'])
clf.fit(data_train[predictors],data_train['Survived'])
accuracy = sum((clf.predict(data_train[predictors]) == data_train['Survived']).astype('float'))/len(data_train)
accuracy1 = sum((rfc.predict(data_train[predictors]) == data_train['Survived']).astype('float'))/len(data_train)
predictions = clf.predict(data_test[predictors])
predictions1 = rfc.predict(data_test[predictors])
predictions2 = (predictions + predictions1)/2.0
predictions2[predictions2 > .5] = 1
predictions2[predictions2 <= .5] = 0
predictions2 = predictions2.astype('int')
submisson = pd.DataFrame({ 'PassengerId' : data_test['PassengerId'], 'Survived': predictions })
submisson.to_csv('titanic-predictions.csv', index = False)
train_pd = clf.predict(data_train[predictors])
train_pd2 = rfc.predict(data_train[predictors])
train_pd3 = (train_pd+train_pd2)/2.0
train_pd3[train_pd3 > .5] = 1
train_pd3[train_pd3 <= .5] = 0
print(sum((train_pd3 == data_train['Survived']).astype('float'))/len(data_train))
print(accuracy)
print(accuracy1)

0.913580246914
0.913580246914
0.948372615039
